In [1]:
import pandas as pd
import numpy as np

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\ML_Projects\\Pred_maintainance-Project'

In [5]:
train = pd.read_csv("artifacts/train.csv")

In [6]:
test = pd.read_csv("artifacts/test.csv")

In [7]:
from sklearn.compose import ColumnTransformer

In [8]:
from sklearn.preprocessing import RobustScaler

In [9]:
from sklearn.decomposition import PCA

In [10]:
from sklearn.pipeline import Pipeline

In [11]:
train.columns

Index(['Engine_no', 'Cycle_no', 'Setting_1', 'Setting_2', 'Setting_3',
       'Fan_inlet_temperature', 'LPC_outlet_temperature',
       'HPC_outlet_temperature', 'LPT_outlet_temperature',
       'Fan_inlet_pressure', 'Bypass_duct_pressure', 'HPC_outlet_pressure',
       'Physical_fan_speed', 'Physical_core_speed', 'Engine_pressure_ratio',
       'HPC_outlet_static_pressure', 'Fuel_flow_ratio', 'Fan_speed',
       'Core_speed', 'Bypass_ratio', 'Burner_fuel_air_ratio', 'Bleed_enthalpy',
       'Required_fan_speed', 'Required_fan_conversion_speed',
       'High_pressure_cool_air_flow', 'Low_pressure_cool_air_flow', 'RUL'],
      dtype='object')

In [1]:
num_cols = ['Engine_no', 'Cycle_no',
       'LPC_outlet_temperature',
       'HPC_outlet_temperature', 'LPT_outlet_temperature',
       'HPC_outlet_pressure',
       'Physical_fan_speed', 'Physical_core_speed', 
       'HPC_outlet_static_pressure', 'Fuel_flow_ratio', 'Fan_speed',
       'Bypass_ratio', 'Bleed_enthalpy',
       'High_pressure_cool_air_flow', 'Low_pressure_cool_air_flow']

In [ ]:
drop_cols = [
    'Setting_1', 'Setting_2', 'Setting_3', 'Fan_inlet_temperature', 'Fan_inlet_pressure', 
    'Bypass_duct_pressure', 'Engine_pressure_ratio', 'Core_speed', 'Burner_fuel_air_ratio', 
    'Required_fan_speed', 'Required_fan_conversion_speed']    


In [13]:
train = train[num_cols]

In [14]:
test = test[num_cols]

In [15]:
train.head()

,Engine_no,Cycle_no,LPC_outlet_temperature,HPC_outlet_temperature,LPT_outlet_temperature,HPC_outlet_pressure,Physical_fan_speed,Physical_core_speed,HPC_outlet_static_pressure,Fuel_flow_ratio,Fan_speed,Bypass_ratio,Bleed_enthalpy,High_pressure_cool_air_flow,Low_pressure_cool_air_flow,RUL
0,1,1,641.82,1589.70,1400.60,554.36,2388.06,9046.19,47.47,521.66,2388.02,8.4195,392,39.06,23.4190,191
1,1,2,642.15,1591.82,1403.14,553.75,2388.04,9044.07,47.49,522.28,2388.07,8.4318,392,39.00,23.4236,190
2,1,3,642.35,1587.99,1404.20,554.26,2388.08,9052.94,47.27,522.42,2388.03,8.4178,390,38.95,23.3442,189
3,1,4,642.35,1582.79,1401.87,554.45,2388.11,9049.48,47.13,522.86,2388.08,8.3682,392,38.88,23.3739,188
4,1,5,642.37,1582.85,1406.22,554.00,2388.06,9055.15,47.28,522.19,2388.04,8.4294,393,38.90,23.4044,187


In [18]:
train["RUL"][train["RUL"] > 128] = 128
test["RUL"][test["RUL"] > 128] = 128

In [20]:
x_train = train.iloc[:, :-1]
y_train = train.iloc[:, -1]
x_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

In [22]:
scaling = RobustScaler()
x_train_scale = scaling.fit_transform(x_train)
x_test_scale = scaling.transform(x_test)

In [23]:
pca = PCA(n_components= 0.95)

x_train_pca = pca.fit_transform(x_train_scale)
x_test_pca = pca.transform(x_test_scale)

In [24]:
from sklearn.svm import SVR

In [26]:
svr = SVR(
    C=5.0, epsilon=0.2, 
         kernel='rbf', 
         degree=3, 
         gamma='auto', 
         coef0=0.0, 
         shrinking=True, 
         tol=0.001, 
         cache_size=500, 
         verbose=False, 
         max_iter=-1
)

In [27]:
svr.fit(x_train_pca, y_train)

SVR(C=5.0, cache_size=500, epsilon=0.2, gamma='auto')

In [28]:
y_pred = svr.predict(x_test_pca)

In [30]:
from sklearn.metrics import r2_score

In [31]:
r2_square = r2_score(y_test, y_pred)

In [33]:
r2_square

0.6751840377644628